# Run the SONAR-netCDF4 v1 convention checker on an EK80 file

In [1]:
import echopype as ep
from echopype.testing import TEST_DATA_FOLDER

from sonarnetcdf4_echopype_checker import cc

In [2]:
import datetime

print(f"echopype version: {ep.__version__}")
print(f"{datetime.datetime.utcnow()} +00:00")

echopype version: 0.7.2.dev65+ga10d22bf
2023-08-27 21:55:40.474044 +00:00


## Convert the target raw file to an EchoData object

In [3]:
# 7/28/23 Note: transmit_type generated by this file contains nan's
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek80/2019118 group2survey-D20191214-T081342.raw", sonar_model='EK80')

# Other, contrasting EK80 files
# Has 1 beam group with a mix of CW & BB channels
ed = ep.open_raw(TEST_DATA_FOLDER / "ek80_bb_with_calibration/2018115-D20181213-T094600.raw", sonar_model='EK80')
# Has 2 beam groups, where group1 is all BB and group2 (by design) is all cW
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek80/Summer2018--D20180905-T033113.raw", sonar_model='EK80')
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek80/D20170912-T234910.raw", sonar_model='EK80')

In [4]:
ed

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains complex backscatter data and other beam or channel-specific data.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

## Test the `Sonar/Beam_group1` group

### Read and process the CDL

In [5]:
target_group = "Sonar/Beam_group1"

In [6]:
conv_check = cc.ConventionCDL(target_group)

### Examine the target group in the converted EchoData object

In [7]:
conv_check.set_ed_group_ds(ed[target_group])

In [8]:
conv_check.ed_group_ds

<xarray.Dataset>
Dimensions:                        (channel: 6, ping_time: 4,
                                    range_sample: 4434, beam: 4)
Coordinates:
  * channel                        (channel) <U22 'WBT 714581-15 ES18' ... 'W...
  * ping_time                      (ping_time) datetime64[ns] 2018-12-13T09:4...
  * range_sample                   (range_sample) int64 0 1 2 ... 4431 4432 4433
  * beam                           (beam) <U21 '1' '2' '3' '4'
Data variables: (12/26)
    frequency_nominal              (channel) float64 1.8e+04 1.2e+05 ... 2e+05
    beam_type                      (channel) int64 1 1 1 65 1 1
    beamwidth_twoway_alongship     (channel) float64 10.2 6.56 ... 5.25 5.64
    beamwidth_twoway_athwartship   (channel) float64 10.31 6.57 ... 6.47 6.06
    beam_direction_x               (channel) float64 nan nan nan nan nan nan
    beam_direction_y               (channel) float64 nan nan nan nan nan nan
    ...                             ...
    transmit_power                 (channel, ping_time) float64 2e+03 ... 105.0
    transmit_duration_nominal      (channel, ping_time) float32 0.001024 ... ...
    slope                          (channel, ping_time) float64 0.5 ... 0.009766
    channel_mode                   (channel, ping_time) int8 1 1 1 1 ... 1 1 1 1
    transmit_type                  (channel, ping_time) <U3 'CW' 'CW' ... 'CW'
    sample_time_offset             (channel, ping_time) float64 0.0 0.0 ... 0.0
Attributes:
    beam_mode:              vertical
    conversion_equation_t:  type_3

### Set the obligation and echopype_mods filters (optional)

In [9]:
conv_check.set_obligation(['M']) # "M", ['M', 'MA'], None, exclude=True

In [10]:
conv_check._get_obligation_vars()

array(['backscatter_r', 'beam', 'beam_direction_x', 'beam_direction_y',
       'beam_direction_z', 'beam_stabilisation', 'beam_type',
       'equivalent_beam_angle', 'non_quantitative_processing',
       'ping_time', 'sample_interval', 'sample_time_offset',
       'transmit_duration_nominal', 'transmit_frequency_start',
       'transmit_frequency_stop', 'transmit_type'], dtype=object)

`echopype_mods` handling is not fully implemented yet

In [11]:
conv_check.set_echopype_mods(None) # None

In [12]:
conv_check.echopype_mods

### Run the tests

#### Presence of expected variables

In [13]:
conv_check.test_vars_presence(test_type="expected")

****Expected variables not found in the EchoData object:


#### Presence of unexpected variables

In [14]:
conv_check.test_vars_presence(test_type="unexpected")

****EchoData variables not found in the CDL:
channel_mode


#### Variable data type

Use two types of data type comparisons: strict (specific) vs generalized.

In [15]:
conv_check.test_vars_datatype(dtype_strict=True)

****Variables with different data type from what is expected:
beam: EchoData type: <U21, CDL type: object
sample_time_offset: EchoData type: float64, CDL type: float32
transmit_type: EchoData type: <U3, CDL type: object
transmit_frequency_start: EchoData type: float64, CDL type: float32
transmit_frequency_stop: EchoData type: float64, CDL type: float32


In [16]:
conv_check.test_vars_datatype(dtype_strict=False)

****Variables with different data type from what is expected:
transmit_type: EchoData type: <U3, CDL type: object


#### Variable dimensionality

In [17]:
conv_check.test_vars_dimensionality()

****EchoData variables with dimensionality different from the CDL:
equivalent_beam_angle: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
beam_direction_x: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
non_quantitative_processing: EchoData dims: (), CDL dims: ('channel', 'ping_time')
beam_direction_z: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
beam_direction_y: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time', 'beam')
beam_stabilisation: EchoData dims: (), CDL dims: ('channel', 'ping_time')
transmit_frequency_start: EchoData dims: ('channel', 'ping_time'), CDL dims: ('channel', 'ping_time', 'beam')
beam_type: EchoData dims: ('channel',), CDL dims: ('channel', 'ping_time')
transmit_frequency_stop: EchoData dims: ('channel', 'ping_time'), CDL dims: ('channel', 'ping_time', 'beam')


#### Attribute presence

In [18]:
conv_check.test_attrs_presence()

****Variable or global missing convention attributes:
transmit_type: Missing EchoData attrs: set()
beam_stabilisation: Missing EchoData attrs: set()


In [19]:
conv_check.test_attrs_presence(global_attrs=True)

****Variable or global missing convention attributes:


#### Attribute values

In [20]:
conv_check.test_attrs_value()

****Variable or global attribute with different values:
non_quantitative_processing.flag_meanings: EchoData value: ['None']
beam_type.long_name: EchoData value: type of transducer (0-single, 1-split)


In [21]:
conv_check.test_attrs_value(global_attrs=True)

****Variable or global attribute with different values:


## Test the `Platform` group

### Read and process the CDL

In [22]:
target_group = "Platform"

In [23]:
conv_check = cc.ConventionCDL(target_group)

### Examine the target group in the converted EchoData object

In [24]:
conv_check.set_ed_group_ds(ed[target_group])

In [25]:
conv_check.ed_group_ds

<xarray.Dataset>
Dimensions:                      (time1: 12, time2: 4, channel: 6)
Coordinates:
  * channel                      (channel) <U22 'WBT 714581-15 ES18' ... 'WBT...
  * time1                        (time1) datetime64[ns] 2018-12-13T09:46:01.3...
  * time2                        (time2) datetime64[ns] 2018-12-13T09:46:00.7...
Data variables: (12/23)
    latitude                     (time1) float64 60.41 60.41 ... 60.41 60.41
    longitude                    (time1) float64 5.312 5.312 ... 5.312 5.312
    sentence_type                (time1) <U3 'GGA' 'GLL' 'RMC' ... 'GLL' 'RMC'
    pitch                        (time2) float64 0.84 0.85 0.8582 0.88
    roll                         (time2) float64 -0.58 -0.58 -0.58 -0.58
    vertical_offset              (time2) float64 0.0 -0.01 -0.01 -0.01
    ...                           ...
    MRU_rotation_y               float64 nan
    MRU_rotation_z               float64 nan
    position_offset_x            float64 nan
    position_offset_y            float64 nan
    position_offset_z            float64 nan
    frequency_nominal            (channel) float64 1.8e+04 1.2e+05 ... 2e+05
Attributes:
    platform_name:       
    platform_type:       
    platform_code_ICES:

### Set the obligation and echopype_mods filters (optional)

In [26]:
conv_check.set_obligation(['M']) # "M", ['M', 'MA'], None, exclude=True

In [27]:
conv_check._get_obligation_vars()

array([], dtype=object)

`echopype_mods` handling is not fully implemented yet

In [28]:
conv_check.set_echopype_mods(None) # None

In [29]:
conv_check.echopype_mods

### Run the tests

#### Presence of expected variables

In [30]:
conv_check.test_vars_presence(test_type="expected")

****Expected variables not found in the EchoData object:


#### Presence of unexpected variables

In [31]:
conv_check.test_vars_presence(test_type="unexpected")

****EchoData variables not found in the CDL:


#### Variable data type

Use two types of data type comparisons: strict (specific) vs generalized.

In [32]:
conv_check.test_vars_datatype(dtype_strict=True)

****Variables with different data type from what is expected:


In [33]:
conv_check.test_vars_datatype(dtype_strict=False)

****Variables with different data type from what is expected:


#### Variable dimensionality

In [34]:
conv_check.test_vars_dimensionality()

****EchoData variables with dimensionality different from the CDL:


#### Attribute presence

In [35]:
conv_check.test_attrs_presence()

****Variable or global missing convention attributes:


In [36]:
conv_check.test_attrs_presence(global_attrs=True)

****Variable or global missing convention attributes:


#### Attribute values

In [37]:
conv_check.test_attrs_value()

****Variable or global attribute with different values:


In [38]:
conv_check.test_attrs_value(global_attrs=True)

****Variable or global attribute with different values:
